In [3]:
import sys
sys.path.append("..")

In [4]:
from last_bbox_for_tracker import LastBBoxForTracker
import ultralytics
import cv2
from pathlib import Path

### Load Ultralytics model

In [ ]:
MODEL_PATH = "models/yolov8n.pt"
model = YOLO(MODEL_PATH)

### Load video

In [ ]:
VIDEO_PATH = "data/videos/sample_video.mp4"
cap = cv2.VideoCapture(VIDEO_PATH)
frame_idx = 0

### Load tracker fallback

In [ ]:
MAX_MISSING_FRAMES = 30
fallback_tracker = LastBBoxForTracker(max_missing_frames=MAX_MISSING_FRAMES)

### Process video with tracker

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(
        frame,
        tracker="bytetrack.yaml",
        persist=True,
        conf=0.3,
        verbose=False,
    )

    detections = []

    if results[0].boxes.id is not None:
        boxes = results[0].boxes.xyxy.cpu().numpy()
        ids = results[0].boxes.id.cpu().numpy().astype(int)

        for bbox, track_id in zip(boxes, ids):
            detections.append(
                (track_id, tuple(map(int, bbox)))
            )

    fallback_tracker.update(frame_idx, detections)

In [ ]:
    for track_id, bbox, is_fallback in fallback.active_tracks(frame_idx):
        x1, y1, x2, y2 = bbox

        color = (0, 165, 255) if is_fallback else (0, 255, 0)

        cv2.rectangle(
            frame,
            (x1, y1),
            (x2, y2),
            color,
            2,
        )

        cv2.putText(
            frame,
            f"ID {track_id}",
            (x1, y1 - 5),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            color,
            2,
        )

    cv2.imshow("Last BBox Fallback", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

    frame_idx += 1